In [ ]:
import os
import gpxpy
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
from geopy.distance import geodesic
from datetime import timedelta
from datetime import datetime

In [ ]:
# Hohenwarte

gpx_file_path = "../data/sample_gpx/VCA_CX.gpx"  # Ersetze mit deinem GPX-Dateipfad
water_path = "../data/plot_data/gis_osm_water_a_free_1.shp"  # Ersetze mit deinem Shapefile-Pfad
halle = gpd.read_file("../data/plot_data/halle.geojson")
osm_landuse = gpd.read_file("../data/plot_data/osm_landuse.geojson")

# Farbzuordnung für bestimmte fclass-Werte
color_map = {
    'forest': '#2E8B57',
    'farmland': '#D2B48C',
    'grass': '#9ACD32',
    'meadow': '#BDB76B',
    'scrub': '#A2CD5A',
    'park': '#66CDAA',
    'recreation_ground': '#20B2AA',
    'orchard': '#FFD700',
    'vineyard': '#8B4513',
    'cemetery': '#999999'
}

# Neue Spalte mit Farben – alle anderen werden NaN (unsichtbar)
osm_landuse['fill_color'] = osm_landuse['fclass'].map(color_map)

# Nur sichtbare Flächen behalten (nur definierte Farben)
osm_landuse_colored = osm_landuse[~osm_landuse['fill_color'].isna()]

#padding für Puffer um GPX definieren, Standart 0.01 - für quadratische Ausgabe höher wählen und dann zuschneiden
padding = 0.1
# Zusätzliches Shapefile z.B. Gewässer OSM laden
addShapefile = True

# GPX-Datei einlesen
with open(gpx_file_path, "r") as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# GPX-Daten extrahieren (Latitude, Longitude, Zeit)
lats, lons, times = [], [], []

# Tracks und Segmente durchlaufen
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            lats.append(point.latitude)
            lons.append(point.longitude)
            times.append(point.time)

# Berechnung des Bounding-Boxes für den GPX-Track
north, south, east, west = max(lats), min(lats), max(lons), min(lons)
#padding = Puffer um den Track
north += padding
south -= padding
east += padding
west -= padding

# Bounding-Box erstellen
bbox = (west, south, east, north)  # (left, bottom, right, top)

# Herunterladen des Straßennetzes mit osmnx
G = ox.graph_from_bbox(
    bbox=bbox,
    network_type="bike",
    simplify=True,
    retain_all=True,
    truncate_by_edge=True,
)

# Kanten des Netzwerks extrahieren
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Distanz berechnen
distance_km = sum(
    geodesic((lats[i], lons[i]), (lats[i + 1], lons[i + 1])).km
    for i in range(len(lats) - 1)
)

# Visualisierung mit matplotlib
fig, ax = plt.subplots(figsize=(10, 10))

# Straßennetz als Hintergrund zeichnen
edges.plot(ax=ax, linewidth=0.3, color="gray", alpha=0.5, zorder=2)

# GPX-Track zeichnen
ax.plot(lons, lats, color="white", linewidth=1, zorder=3)
#halle.plot(ax=ax, linewidth=1.5, color="#FF6984",linestyle='--', alpha=1, zorder=1)

if addShapefile:
    water = gpd.read_file(water_path)

    # Optional: Falls du bestimmte Farben und Stile für das Shapefile festlegen möchtest:
    #shapefile.plot(ax=ax, color="grey", alpha=1, linewidth=2, zorder=10)
    water.plot(ax=ax, color="#87CEFA", alpha=0.7, linewidth=2, zorder=1)

osm_landuse_colored.plot(ax=ax, color=osm_landuse_colored['fill_color'], edgecolor='none', linewidth=0.2, alpha=0.5, zorder=2)

# Hintergrund setzen
ax.set_facecolor("#333333")
fig.patch.set_facecolor("#333333")

# Achsen entfernen
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim([west, east])
ax.set_ylim([south, north])

# Plot anzeigen
plt.show()

# Generiere den Zeitstempel im Format "YYYY-MM-DD_HH-MM-SS"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


output_directory = "../data/export/"
output_filename = os.path.join(output_directory, f"01_single_track_{timestamp}.png")

# Speichern der Grafik mit dem neuen Dateinamen
fig.savefig(output_filename, dpi=1080, bbox_inches="tight", facecolor="black")
print(f"Bild gespeichert als: {output_filename}")